In [ ]:
# 12-1 并发、并行、同步、异步、阻塞、非阻塞

并发：一个时间段内，有几个程序在同一个cpu上运行，但是任意时刻只有一个程序在cpu上                            一段时间内                  数量取决于                       可以追求高并发     比如一个时间段内，某个请求的响应很快，则能处理的请求就多，就达到高并发
并行：在任意时刻点上，有多个程序同时运行在多个cpu上    多个cpu，每个cpu独立运行自己的程序，之间互不干扰     在任意时刻，同时运行的程序   并行的个数是和cpu核心数一致的    不可以达到高并行 核心数有限

以下的概念必须涉及到io操作才会出现
同步：代码调用IO操作的时候，必须等待IO操作完成才返回的调用方式
异步：代码调用IO操作的时候，不必等待IO操作完成就返回的调用方式   多线程时典型的异步操作        同步和异步是一种消息通信机制，io操作相当于消息，同步是消息一发出，就收到一个返回结果
阻塞：指调用函数的时候当前线程被挂起
非阻塞：指调用函数时候当前线程不会被挂起，而是立即返回                                        阻塞和非阻塞是一种函数调用机制
 

    
异步只有配合非阻塞的才有意义，
同步只能让调用者去轮询是否继续，而异步会有提醒，
异步的阻塞是没有意义的，           https://www.jb51.net/article/156545.htm


In [ ]:
# 12-2 IO 多路复用 (select、poll 和 epoll)

C10K问题： 如何在一颗1GHz CPU,2G内存，1gbps网络环境下，让单台服务器同时为1万个客户提供FTP服务

Unix下五中I/O模型

阻塞式I/O      等待io的过程当中，cpu被大量浪费 阻塞式I/O的过程中，cpu是大量空闲的
非阻塞式I/O
I/O多路复用    select监听多个
信号驱动式I/O
异步I/O   （POSIX的aio_系列函数）   aio_read



UNIX 网络编程

In [ ]:
# 12-3 select+回调+事件循环获取html-1



In [ ]:
# 12-5 回调之痛     回调模式 正在单线程中实现并发

In [15]:
# 12-6 协程是什么   使用回调模式虽然能解决并发问题，但是使用复杂


# C10M问题： 如何利用8核心cpu，64G内存，在10gbps的网络上保持1000万并发连接     涉及到 协程

'''
实现单线程下并发，出现的问题：
1. 回调模式编码复杂度高
2. 同步编程的并发性不高
3. 多线程编程需要线程间的同步，使用锁的机制 但是锁机制会降低并发性能

1. 采用同步的方式去编写异步代码
2. 使用单线程去切换任务
        1. 线程是由操作系统切换的，单线程切换意味着我们需要程序员自己去调度任务
        2. 不再需要锁（使用锁是为了完成线程间的同步，因为是单线程下不需要锁），并发性高     线程代价比较大，需内存，切换过程比较慢， 函数之间的调用（函数调用，比线程之间的切换性能高）  如果单线程内切换函数，则性能远高于线程切换，并发性更高
'''
# 传统函数调用，过程 A->B->C 每一个函数都是有一个栈的，一旦运行一次后会退出（运行后直到返回为止），退出后就与该函数的状态没有关系了  
#我们需要一个可以暂停的函数，并且可以在适当的时候恢复该函数的继续执行
#出现了协程 -> 有多个入口的函数， 可以暂停的函数， 可以暂停的函数(可以向暂停的地方传入值) ---  使用生成器

def gen_fun():
    #1.可以产出值  2.可以接收值（调用方传递进来的值）
    html = yield "http://www.baidu.com"
    print("打印：",html)
    yield 2
    yield 3
    return "tt"

if __name__ == "__main__":
    
    gen = gen_fun()
    # 第一次调用生成器的时候，不能传递一个非None的值 TypeError: can't send non-None value to a just-started generator
    # 所以如果想使用send  必须是gen.send(None)  因为send是将值传递给 yield，而生成器开始的时候没有到yield
    # 在调用send发送非none值之前，我们必须启动一次生成器， 方式有两种1. gen.send(None), 2. next(gen)
    print(next(gen))  
    html = 'html1'
    print(gen.send(html))  # send 方法可以传递值进入生成器内部，同时还可以重启生成器执行到下一个yield位置（next()的功能）


http://www.baidu.com
打印： html1
2


In [22]:
# 生成器的close方法
def gen_func():
    #1. 可以产出值， 2. 可以接收值(调用方传递进来的值)
    yield "http://projectsedu.com"
    print("fdsadf")
    yield 2
    yield 3
    return "bobby"

if __name__ == "__main__":
    gen = gen_func()
    print(next(gen))
    gen.close()   # 会在生成器对象方法的挂起处抛出一个GeneratorExit异常。GeneratorExit异常产生后，系统会继续把生成器对象方法后续的代码执行完毕。
    print("bobby")  # 代码正常执行

    #GeneratorExit是继承自BaseException， Exception



http://projectsedu.com
bobby


In [41]:
def gen_func():
    #1. 可以产出值， 2. 可以接收值(调用方传递进来的值)
    try:
        yield "http://projectsedu.com"
    except GeneratorExit :   # close抛出的异常为GeneratorExit，GeneratorExit是继承自BaseException， 而没有继承Exception     
        print("fdsa")
    yield 2
    yield 3
    return "bobby"

if __name__ == "__main__":
    gen = gen_func()
    print(next(gen))
    gen.close()    # 注意的是close方法只是在被挂起的地方抛出一个异常，意味着生成器内部之后的代码都无法执行，但是在没有显式捕获异常的时候，会内部消化（不会向主程序抛出）   
    print("bobby")  # 所以最好的做法就是不要去处理这个
    #GeneratorExit是继承自BaseException， 而没有继承Exception

    

  

http://projectsedu.com
fdsa


RuntimeError: generator ignored GeneratorExit

In [ ]:
总结：生成器没有执行完时就强行终止会报GeneratorExit异常
如果该异常没有被显式捕获，生成器对象也不会把该异常向主程序抛出。因为GeneratorExit异常定义的初衷，是方便开发者在生成器对象调用结束后定义一些收尾的工作，如释放资源等。
产生GeneratorExit异常后。并不会报错，但是若此时再次调用生成器对象就会激活这个异常
GeneratorExit异常的产生意味着生成器对象的生命周期已经结束。因此，一旦产生了GeneratorExit异常，生成器方法后续执行的语句中，不能再有yield语句，否则会产生RuntimeError

In [50]:
# throw方法
def gen_func():
    try:
        yield "http://projectsedu.com"
    except Exception as e:
        pass
    yield 2
    yield 3
    return "bobby"

if __name__ == "__main__":
    gen = gen_func()
    print(next(gen))
    re = gen.throw(Exception, "download error")
    print(re)
    print(next(gen))


http://projectsedu.com
2


throw()：它的实现手段是通过向生成器对象在上次被挂起处，抛出一个异常。<font color='red'>之后会继续执行生成器对象中后面的语句，直至遇到下一个yield语句返回</font>。要设置异常捕获，否则会报错，生成器无法继续执行。如果在生成器对象方法执行完毕后，依然没有遇到yield语句，抛出StopIteration异常。
使用场景：一般就是需要有特殊操作的时候，又不想打断生成器，又或是相对生成器的中间值进行修改等
</br>
</br>
</br>
throw()方法和close()方法的区别在于使用：使用throw()，在上次挂起的地方抛出异常，若异常被捕获，会继续执行生成器对象中后面的语句，直至遇到下一个yield语句返回（即throw()是有返回值的，就相当于做了两件事，next()语句的功能和在上一次挂起的地方添加功能），异常被捕获之后可以继续往下执行
</br>
而close()方法则是相当于宣告生成器结束了，但是close()不会执行生成器，宣告生成器结束了之后，会直接执行生成器函数外部之后所有的方法。



In [53]:
# 12-8 生成器进阶-yield from-1

#python3.3新加了yield from语法
from itertools import chain      
# chain方法 将所有可迭代的对象组合到一起参加for循环 

my_list = [1,2,3]
my_dict = {
    "bobby1":"http://projectsedu.com",
    "bobby2":"http://www.imooc.com",
}

for value in chain(my_list, my_dict, range(4,10)):
    print(value)


1
2
3
bobby1
bobby2
4
5
6
7
8
9


In [55]:
# 我们可以手动实现这个原理
def my_chain(*args, **kwargs):
    for iterable in args:
        for value in iterable:
            yield value   # 使用生成器也完成了该方法

for value in my_chain(my_list, my_dict, range(4,10)):
    print(value)

1
2
3
bobby1
bobby2
4
5
6
7
8
9


In [56]:
# 使用yield from 来实现上面的功能
# 用法： yield from iterable         使用yield from 后面跟可迭代对象，可以将可迭代对象中的值返回，不需要再使用for循环了
def my_chain2(*args, **kwargs):
    for iterable in args:
        yield from iterable  # 使用yield from 完成了

for value in my_chain2(my_list, my_dict, range(4,10)):
    print(value)

1
2
3
bobby1
bobby2
4
5
6
7
8
9


In [66]:
# yield from iterable 做的事远不止与代替了 for 循环后 yield

def g1(iterable):
    yield iterable

def g2(iterable):
    yield from iterable  # 会循环遍历iterable中的值
    
for value in g1(range(10)):
    print(value)
for value in g2(range(10)):
    print(value)    

range(0, 10)
0
1
2
3
4
5
6
7
8
9


In [63]:
def g1():
    yield from gen()     #gen 是子生成器    此处不能是函数名，而必须是函数调用的形式，因为生成器只有被调用了才得到的生成器对象（yield from 后面必须是可迭代对象）

def main():
    g = g1()
    g.send(None)     #  此时send的发送值直接发送到gen了   

#1. main 调用方   g1(委托生成器)     gen 子生成器
#1. yield from会在调用方main与子生成器gen之间建立一个双向通道

'''
    说明:一般情况下，函数的调用，这里如果当做普通的函数调用来讲的话是g1函数接收到gen函数的值，然后将结果返回到main函数，而main函数中send值时应该被g1接收到，但是因为使用了yield from，值将直接传给gen
    yield from会在调用方main与子生成器gen之间建立一个双向通道，      此时main 和 gen 之间的将会直接进行信息传递 相当于在main中传递值直接给gen，gen的值直接给main
'''


In [102]:
def gen():
    for i in range(10):
        yield i
    return "ddd"

def g1():
    a = yield from gen()
    print("a",a)

# def main():
#     g = g1()
#     print(next(g)) 
    

In [103]:
g = g1()
for i in g1():
    print(i)

0
1
2
3
4
5
6
7
8
9
a ddd


In [ ]:
# 最后一次的gen的返回值被yield from 捕获

In [2]:
# 委托生成器模式

final_result = {}


def sales_sum(pro_name):
    """子生成器"""
    total = 0
    nums = []
    while True:
        x = yield
        if not x:  # main中的for循环下的m.send(None)在传递给x之后，循环退出，此时m.send(None)会像next()方法那样继续的调用，此时会异常，StopIteration 外接 返回值，
            break
        print(pro_name+"销量: ", x)
        total += x
        nums.append(x)
    return total, nums  # 该返回值被middle中的yield from 捕获，


def middle(key):
    '''委托生成器'''
    while True:
        final_result[key] = yield from sales_sum(key)  # 上面的生成器出StopIteration错误 并且返回 return的值，而StopIteration异常的捕获和return的接收，都可以直接交给yield from 处理
        print(key+"销量统计完成！！.")



def main():
    '''主函数'''
    data_sets = {
        "bobby牌面膜": [1200, 1500, 3000],
        "bobby牌手机": [28, 55, 98, 108],
        "bobby牌大衣": [280, 560, 778, 70],
    }
    for key, data_set in data_sets.items():
        print("start key:", key)
        m = middle(key)
        m.send(None)  # 预激middle协程       ----   m=middle(key)只是得到了一个生成器对象，使用的话 需要next(m) 或者m.send(None)才能使用
        for value in data_set:
            m.send(value)   # 给协程传递每一组的值
        m.send(None)

        # print("the last :", m.send(23324))   # 通过此段代码测试后发现，在m.send(None)之后，sales_sum异常，middle继续执行后又碰到了yield from sales_sum(key)，此时又进入sales_sum子生成器，建立了连接
    print("final_result:", final_result)


if __name__ == '__main__':
    main()

start key: bobby牌面膜
bobby牌面膜销量:  1200
bobby牌面膜销量:  1500
bobby牌面膜销量:  3000
bobby牌面膜销量统计完成！！.
start key: bobby牌手机
bobby牌手机销量:  28
bobby牌手机销量:  55
bobby牌手机销量:  98
bobby牌手机销量:  108
bobby牌手机销量统计完成！！.
start key: bobby牌大衣
bobby牌大衣销量:  280
bobby牌大衣销量:  560
bobby牌大衣销量:  778
bobby牌大衣销量:  70
bobby牌大衣销量统计完成！！.
final_result: {'bobby牌面膜': (5700, [1200, 1500, 3000]), 'bobby牌手机': (289, [28, 55, 98, 108]), 'bobby牌大衣': (1688, [280, 560, 778, 70])}


In [3]:
## Python3.5以前都是使用的yield生成器来完成协程的’
# Python3.5之后，为了使语义变得更加明确，就引入了async和await关键词用于定义原生的协程
'''
在写协程的时候尽量使用async 和 await 关键字来实现  这样会少很多歧义（使用yield实现生成器的时候会分不清是生成器还是协程）
可以使用生成器去理解协程
在原生协程的Python内部实现任然使用的是生成器原理    （一个生成器就是一个协程？姑且这么理解吧）
'''
async def download(url):
#     yield 1
    return "bobby"         

async def download_url(url):   # python 3.5 之后定义的 加强了协程的定义，使得await只能在被async修饰的函数中出现，且有await关键字的时候不能出现yield关键字
    # dosomething 
    html = await download(url)   # 相当于 yield from       python 3.5 之后定义的 
    return html  

if __name__ == "__main__":
    coro = download_url("http://www.ic.com")
    coro.send(None)   # 原生协程必须使用 send(None)来启动    而不能使用next()

    

StopIteration: bobby

In [10]:
import types

@types.coroutine      
def downloader(url):
    yield "bobby"
    
async def download_url(url):
    #dosomethings
    html = await downloader(url)     # await的语义效果就是 将控制权交出并等待结果返回
    return html

if __name__ == "__main__":
    coro = download_url("http://www.imooc.com")
    # next(None)
    coro.send(None)
    